<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/RAG_hybrid_Mistral_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)
!pip install -q optimum --progress-bar off
!pip install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  --progress-bar off # Use cu117 if on CUDA 11.7
# We need specific transformer to make mistral work
!pip install -q git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79 --progress-bar off

!pip install -qU langchain Faiss-gpu sentence-transformers
!pip install -q jq # for json loader to work
!pip install ctransformers[gptq] #To use CTransformer from langchain and load gptq model

# !pip install -qU trl Py7zr
!pip install -q rank_bm25
!pip install -q PyPdf

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 12.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 656.0/656.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14

In [2]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import langchain
from langchain.storage import InMemoryStore
from langchain.cache import InMemoryCache
from langchain.llms import CTransformers

config = {'max_new_tokens': 1024, 'temperature': 0.1, 'repetition_penalty': 1.1}

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.1-GPTQ"

llm = CTransformers(model=model_name_or_path,
                    config=config)

# Have cache
langchain.llm_cache = InMemoryCache()

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [25]:
## TEST Prompts

PROMPT_TEMPLATE = """<s> [INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Remove and replace any personal information or PII from the context, and the answer
- Remove and replace any placeholder text on the answer (e.g. "Answer:", "<PERSON>", etc.)
- Use Markdown formatting
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language

If you're unsure about the answer:
- For general knowledge about immigration, relocation, settle-in, cultural integration or similar fields related queries, share your amazing knowledge to answer the query but make sure to suggest them to verify with the user's agent
- For feedbacks or suggestions, thank them and share the respond accordingly
- For questions that are specific about their own or family member's immigration case progress, reply with "OUT_OF_CONTEXT" to avoid providing incorrect information
- For unrelated questions which isn't generic to answer, reply with "OUT_OF_CONTEXT" to avoid hallucinating the answer
- For questions that cannot be answer by the context say "OUT_OF_CONTEXT". [/INST] </s>

[INST] Question:  Should I need TRP?

Context:
- Your passport must have at least 2 empty pages and be valid for 3 months after the end date of the visa you're applying for. \nWe always apply for a maximum duration of a visa (1 year) thus if your passport expires earlier than this, then you will be issued a visa for a shorter duration. Therefore we strongly advise to renew your passport before applying for a visa so we may apply for maximum duration.
- If you're changing jobs or your purpose of stay in Estonia changes (for example from studies to employment) then it is required to apply for a new TRP. If the application is submitted while your current residence permit is valid, you will have legal grounds to stay in Estonia until the new one is issued. As you can sign digitally, we will submit your application and documents to the Police Board electronically.

Answer: [/INST]"""

llm(PROMPT_TEMPLATE)


" Based on the context provided, it appears that you are asking about whether or not you need a TRP (Temporary Residence Permit) when applying for a visa to Estonia. \n\nIf you are changing jobs or your purpose of stay in Estonia changes, then it is required to apply for a new TRP. This means that if you are currently in Estonia with a valid residence permit and plan to change your job or purpose of stay, you will need to apply for a new TRP before the end of your current permit's validity period.\n\nIt is important to note that if you submit your application while your current residence permit is still valid, you will have legal grounds to stay in Estonia until the new one is issued. Additionally, it is recommended to renew your passport before applying for a visa so that you may apply for the maximum duration of a visa (1 year).\n\nIf you have any further questions or concerns about the TRP application process, please feel free to reach out to your assigned agent for assistance."

In [26]:
from langchain.storage import InMemoryStore
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings

# In our implementation we have used uses the local file system for storing embeddings and FAISS vector store for retrieval.
# store = LocalFileStore("./cache/")

# We can also set up inmemory cache
# NOTE: we used this as we are more familiar with it
store = InMemoryStore()

embed_model_id="thenlper/gte-large"

# Under the hood HuggingFaceEmbeddings is using sentence-transformer
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)

# Here we will leverage a CacheBackedEmbeddings to prevent us from re-embedding similar queries over and over again.
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Loading the tokenizer from the `special_tokens_map.json` and the `added_tokens.json` will be removed in `transformers 5`,  it is kept for forward compatibility, but it is recommended to update your `tokenizer_config.json` by uploading it again. You will see the new `added_tokens_decoder` attribute that will store the relevant information.


In [ ]:
from langchain.document_loaders import JSONLoader

# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["country"] = record.get("country")
    metadata["answer"] = record.get("answer")

    return metadata


loader = JSONLoader(
    file_path='/content/drive/MyDrive/datasets/qna-clean.json',
    jq_schema='.[]',
    content_key="question",
    metadata_func=metadata_func
)

faq_docs = loader.load()
faq_docs

In [28]:
from langchain.vectorstores import FAISS

# Create VectorStore
vectorstore = FAISS.from_documents(faq_docs, embedder)

In [8]:
# Testing embeddings

query = "When should I move out from Estonia?"
embedding_vector = core_embeddings_model.embed_query(query)
print(len(embedding_vector))
#
docs_resp = vectorstore.similarity_search_by_vector(embedding_vector,k=5)
#
for page in docs_resp:
  print(page.page_content)
  print("\n")

1024
I am due to run out of my Schengen allowance and my residence permit has not been issued yet. Can I stay in Estonia?


I am due to run out of my Schengen visa-free days and my residence permit has not been issued yet. Can I stay in Estonia?


How do I obtain a Temporary Right of Residence in Estonia?


How can I find out how long I am authorized to stay in Estonia?


How can I find out how long I am authorized to stay in Estonia?




In [35]:
_retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': 'Estonia'} })
_retriever.get_relevant_documents("When should I move out from Estonia?")

[Document(page_content='I am due to run out of my Schengen visa-free days and my residence permit has not been issued yet. Can I stay in Estonia?', metadata={'source': '/content/drive/MyDrive/datasets/qna-clean.json', 'seq_num': 35, 'country': 'Estonia', 'answer': "No, a pending Estonian residence permit does not grant you legal stay in Estonia, which means you have to either apply for a visa or leave the country. This is why it's very important to monitor your visa-free or visa days to make sure can stay in Estonia while the permit is processing.\nIf you're applying for your second residence permit (ie you have/had an Estonian residence permit when submitting a new application), you can stay in Estonia while the Police Board is processing the application."}),
 Document(page_content='How do I obtain a Temporary Right of Residence in Estonia?', metadata={'source': '/content/drive/MyDrive/datasets/qna-clean.json', 'seq_num': 326, 'country': 'Estonia', 'answer': 'As an EU citizen, you are

In [29]:
from langchain.retrievers import BM25Retriever

# Sparse
bm25_retriever = BM25Retriever.from_documents(faq_docs)
bm25_retriever.k=5


In [30]:
from langchain.retrievers import EnsembleRetriever

# Hybrid search
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k":5})
ensemble_retriever = EnsembleRetriever(retrievers=[bm25_retriever,faiss_retriever],
                                       weights=[0.5,0.5])

In [48]:
from langchain import PromptTemplate

PROMPT_TEMPLATE = """<s> [INST] You act as a helpful immigration and relocation agent, responding to questions based on provided context on behalf of the user's assigned agent.

Follow these guidelines for the answer:
- Only answer information related to immigration, relocation, settle-in, cultural integration or similar fields related queries.
- Remove and replace any personal information or PII from the context, and the answer
- Remove and replace any placeholder text on the answer (e.g. "Answer:", "<PERSON>", etc.)
- Use Markdown formatting
- Summarize the answer and make it clear, concise, grammatically correct and direct like a human agent would
- Don't disclose yourself as AI, automated tool or bot, answer like the user's agent would
- Never suggest contacting outside Jobbatical platform. Rewrite the answer to contact the user's agent instead
- write the answer using english language

If you're unsure about the answer:
- For feedbacks or suggestions, thank them and share the respond accordingly
- For questions that are specific about their own or family member's immigration case progress, reply with "OUT_OF_CONTEXT" to avoid providing incorrect information
- For unrelated questions which isn't generic to answer, reply with "OUT_OF_CONTEXT" to avoid hallucinating the answer
- For questions that cannot be answer by the context say "OUT_OF_CONTEXT". [/INST] </s>

[INST] Question: {question}

Context:
{context}

Answer: [/INST]"""
#
input_variables = ['context', 'question']
#
custom_prompt = PromptTemplate(template=PROMPT_TEMPLATE,
                            input_variables=input_variables)

In [53]:
import langchain
from langchain.chains import LLMChain
from langchain.callbacks import StdOutCallbackHandler

def get_documents_as_context(query, retriever):
    docs = retriever.get_relevant_documents(query)
    inputs = [f"- {doc.metadata['answer']}" for doc in docs]
    context = "\n".join(inputs)
    return context

def _inference(question, context):
  # Log genereated info
  handler = StdOutCallbackHandler()
  # We use LLMChain + manual retrieval because the answers for our question is inside metadata.
  # There no easyway to do it in Langchain.
  llm_chain = LLMChain(
      llm=llm,
      prompt=custom_prompt,
      verbose=True,
      callbacks=[handler]
  )

  return llm_chain.predict(context=context, question=query)

# To print the final prompt
  langchain.debug = True

# W/o hybrid search

In [54]:
%%time

query = "What is short-term employment registration?"
country = "Estonia"

retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': country} })
context = get_documents_as_context(query, retriever)

_inference(question=query, context=context)



> Entering new LLMChain chain...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "context": "- This step is required so you may legally start working for another company than that for which your previous TRP was issued for, or where you worked before on the basis of a visa.\n- Generally, STER may be registered for up to 365 days within a 455-day period. In some types of employment, STER may be extended (for example, top specialist, ICT specific role).\n- Once we have submitted the paperwork on your employer's behalf, the Police Board will review the application in 2-15 working days, and issue a confirmation as well as an Estonian ID code for you. This will be sent to you as well as us, and unfortunately only in Estonian. We will upload the file, fill your Estonian ID code on the platform, and notify you of the next steps.\n- Once you receive a temporary residence permit, STER is terminated. That's because a temporary residence permit allows you to both reside and 

" Short-term employment registration (STER) is a process required for legally starting work with another company or where you worked before on the basis of a visa. It can be registered for up to 365 days within a 455-day period, and may be extended for certain types of employment. Once the paperwork has been submitted by your employer, the Police Board will review the application in 2-15 working days and issue a confirmation as well as an Estonian ID code. This will be sent to you and us, but only in Estonian. The period marked on the registration confirmation is the time during which you're allowed to work in Estonia with a specific employer, starting at any point within that period. It's important to note that you're not allowed to work in Estonia before or after the time period indicated on STER confirmation."

In [55]:
%%time

query = "when should I receive my residence permit card?"
country = "Estonia"

retriever = vectorstore.as_retriever(search_kwargs={"k":5, 'filter': {'country': country} })
context = get_documents_as_context(query, retriever)

_inference(question=query, context=context)



> Entering new LLMChain chain...
[chain/start] [1:chain:LLMChain] Entering Chain run with input:
{
  "context": "- By law, the processing of your application can take up to 2 months from submission, but the Police Board has the right to extend the deadline if necessary. Additional time for processing can take up to a month.\n- The Estonian residence permit can be either temporary or long-term.\nThere are several possible grounds for a temporary residence permit, including employment, business, settling with a spouse/parent, etc. It's issued for up to 5 years and can be extended for up to 10 years. A long-term residence permit becomes an option once you've been living in Estonia permanently for at least 5 years.\nOnce you have your residence permit, you can reside and work in Estonia without a long-term visa. The Estonian residence permit also allows you to travel within the Schengen area without a visa, carrying your residence permit and national passport. Please note that if your em

" Based on the provided context, it seems that the processing of your residence permit application can take up to 2 months from submission, with additional time for processing possible if necessary. Once your application has been processed and approved, you will receive a temporary residence permit card which allows you to reside and work in Estonia without a long-term visa. The validity of your temporary residence permit depends on your employment contract and is up to the Police Board to decide. It could be anywhere between 1 and 5 years.\n\nIt's important to note that once you receive a temporary residence permit, STER (short-term employment registration) will be terminated. This means that you no longer need to have a short-term employment registration in order to reside and work in Estonia.\n\nIf you have any further questions or concerns about your residence permit application or the process of receiving your residence permit card, please feel free to contact your assigned agent 